In [1]:
import lib.notebook_helpers as nh
import baseline.estimators as estimators
import nncf.run_model_test as nn_test
from nncf.evaluate_recall import evaluate_model_recall
import os
import pickle
import scipy.sparse as sp
from nncf.evaluate_recall import get_preds
from importlib import reload
import numpy as np

TEST_PART_SIZE = 2000

Using TensorFlow backend.


In [9]:
reload(estimators)

<module 'baseline.estimators' from '/home/david/Documents/proj/tag_recommendation/baseline/estimators.py'>

## Load dataset:

In [3]:
test_set_path = os.path.join('data/test_tag_dataset.pkl')
with open(test_set_path, 'rb') as f:
    X, y, mlbx, mlby, val_y, test_y = pickle.load(f)
    y = y.tocsr()
    X = sp.csr_matrix(X)
    test_y = test_y.toarray()

## Train base model and make predictions

In [10]:
base_model = estimators.BaselineModel(factors=1000,
                                regularization=7,
                                iterations=15,
                                show_progress=False,
                                n=3,
                                content_scale_factor=0.0424,
                                alpha=2.17)

In [11]:
base_model.fit(X, y)

BaselineModel(alpha=2.17, content_scale_factor=0.0424, factors=1000,
              filter_seen=True, iterations=15, n=3, regularization=7,
              show_progress=False)

In [6]:
y_pred_base = base_model.predict(X[:TEST_PART_SIZE], start_index=0)

## Load pretrained nn model and make predictions

In [4]:
nn_model_runfile_path = "nncf/runs/several_runs/runu4.yml"

params = nn_test.read_params(nn_model_runfile_path)

nn_model = nn_test.build_model(params, y.shape, X.shape[1])

In [22]:
y_pred_nn = get_preds(nn_model, y[:TEST_PART_SIZE], 3, False, 0, X[:TEST_PART_SIZE].toarray())

## Check results

In [24]:
def test_both(test_y, y_pred_base, y_pred_nn, mlbx, mlby, y, X):
    from random import randint
    index = randint(0, len(test_y) - 1)
    
    
    print("Start user tags: " + str(mlby.inverse_transform(y)[index]))
    print("Start autotags: " + str(mlbx.inverse_transform(X)[index]))
    print("")
    print("Pred goal: " + str(mlby.inverse_transform(test_y)[index]))
    print("")
    print("Base model predicted: " + str(mlby.inverse_transform(y_pred_base)[index]))
    print("")
    print("NN model predicted: " + str(mlby.inverse_transform(y_pred_nn)[index]))

In [ ]:
test_both(test_y, y_pred_base, y_pred_nn, mlbx, mlby, y, X)